In [8]:
import os
import json
from ImageElement import *

print_info = True

coco_dataset_path = 'COCO_dataset'
yolo_dataset_path = 'YOLO_dataset'
coco_annotations_path = os.path.join(coco_dataset_path, 'annotations')
coco_images_path = os.path.join(coco_dataset_path, 'images')

In [9]:
# Проверяем наличие датасета
if not os.path.exists(coco_dataset_path):
    raise FileNotFoundError(f"Папка с COCO датасетом '{coco_images_path}' не найдена. ")


# Проверяем наличие папки с изображениями
if not os.path.exists(coco_images_path):
    raise FileNotFoundError(f"Папка с изображениями '{coco_images_path}' не найдена. "
                            f"Убедитесь, что вы загрузили разметку COCO так, чтобы имелась папка со всеми изображениями.")

# Проверяем, существует ли папка annotations
if not os.path.exists(coco_annotations_path):
    raise FileNotFoundError(f"Папка с json файлами '{coco_annotations_path}' не найдена.")



list_of_image_elements = []
list_of_images_path = []

# Получаем список всех файлов в папке annotations
annotation_files = os.listdir(coco_annotations_path)

# Проходим по файлам аннотаций
for annotation_file in annotation_files:
    # Парсим название файла изображения из файла аннотации
    type_data = os.path.splitext(annotation_file)[0].split('_')[-1]
    json_file_path = os.path.join(coco_annotations_path, annotation_file) # путь к json файлу

    # открытие coco json
    with open(json_file_path, 'r') as f:
        coco_data = json.load(f)

    # Получаем список изображений из JSON
    images = coco_data['images']

    # Создаем словарь с информацией о классах
    coco_categories = coco_data['categories']
    categories_dict = {category['id']-1: category['name'] for category in coco_categories}

    # Выводим информацию
    if print_info:
            print(f'Осуществляется обработка {annotation_file}')
            print(f'Имеющиеся классы: {categories_dict}')
            print('-----------------\n')

    #### Дополнительная прверка на наличие всех файлов изображений
    # Получаем список файлов изображений, для которых есть разметка в COCO
    annotated_images = set([entry['file_name'] for entry in coco_data['images']])

    # Получаем список файлов в папке с изображениями
    all_images = set(os.listdir(coco_images_path))

    # Проверяем, что все изображения из COCO размечены
    if not annotated_images.issubset(all_images):
        missing_images = annotated_images - all_images
        raise FileNotFoundError(f"Некоторые изображения, для которых есть разметка в {json_file_path}, отсутствуют в папке с изображениями. "
                                f"Отсутствующие изображения: {missing_images}")
        
    # Проходим по изображениям и считываем аннотации
    for image in images:
        image_id = image['id']
        file_name = image['file_name']
        path_image_initial = os.path.join(coco_images_path, file_name)
        
        # Находим соответствующие аннотации для изображения
        list_of_lists_annotations = [ann['segmentation'] for ann in coco_data['annotations'] if ann['image_id'] == image_id]
        annotations = [sublist[0] for sublist in list_of_lists_annotations]
        classes = [ann['category_id']-1 for ann in coco_data['annotations'] if ann['image_id'] == image_id]

        # Создаем объект класса ImageElement:
        element = ImageElement(
                path_image_initial=path_image_initial,
                path_label_initial=json_file_path,
                img_width=image['width'],
                img_height=image['height'],
                image_id=image_id,
                type_data=type_data.lower(),
                path_label_final=os.path.join(yolo_dataset_path, type_data.lower(),
                                            'labels', os.path.splitext(file_name)[0]+'.txt'),
                path_image_final=os.path.join(yolo_dataset_path, type_data.lower(),
                                            'images', file_name),
                classes_names=[categories_dict[cl] for cl in classes],
                classes_ids=classes,
                point_list=annotations,
                )
        list_of_image_elements.append(element)
        list_of_images_path.append(file_name)

        # Вывод информации об ImageElement при необходимости
        if print_info:
            print(element)

### Проверка на присутвие всех изображений в папке images 
# Получаем список файлов в папке
files_in_folder = set(os.listdir(coco_images_path))

# Проверяем, что все файлы из списка присутствуют в папке
missing_files = set(list_of_images_path) - files_in_folder
extra_files = files_in_folder - set(list_of_images_path)

# Выводим уведомление
if missing_files:
    print(f"Отсутствующие файлы в папке {coco_images_path}: {missing_files}")

if extra_files:
    print(f"Лишние файлы в папке {coco_images_path}: {extra_files}")

Осуществляется обработка instances_Train.json
Имеющиеся классы: {0: 'car', 1: 'peple'}
-----------------

ImageElement info:
 - path_image_initial: COCO_dataset\images\1.jpg
 - path_label_initial: COCO_dataset\annotations\instances_Train.json
 - img_width: 2048
 - img_height: 1330
 - image_id: 1
 - type_data: train
 - path_label_final: YOLO_dataset\train\labels\1.txt
 - path_image_final: YOLO_dataset\train\images\1.jpg
 - classes_names: []
 - classes_ids: []
 - point_list: []

ImageElement info:
 - path_image_initial: COCO_dataset\images\2.jpg
 - path_label_initial: COCO_dataset\annotations\instances_Train.json
 - img_width: 1100
 - img_height: 733
 - image_id: 2
 - type_data: train
 - path_label_final: YOLO_dataset\train\labels\2.txt
 - path_image_final: YOLO_dataset\train\images\2.jpg
 - classes_names: ['peple', 'peple', 'car', 'car']
 - classes_ids: [1, 1, 0, 0]
 - point_list: [[660.99, 438.73, 636.03, 366.48, 639.98, 352.03, 692.52, 358.6, 696.46, 443.98], [747.69, 546.84, 738.5, 3